# Infonet Security HW3
#### Harris A. Ransom
#### 10/30/2024

## Imports

In [50]:
from math import gcd, sqrt
import secrets
import sympy
import numpy as np

## 1. Auxiliary Functions

In [51]:
# Extended Euclidean Algorithm
def gcdExtended(a, b):
	# Base Case
	if a == 0:
		return b, 0, 1
	
	gcd, x1, y1 = gcdExtended(b % a, a)
		
	# Update x and y using results of recursive call
	x = y1 - (b//a) * x1
	y = x1
	return gcd, x, y

# Efficient modular exponentiation
def exponentiation(m, e, n):
	# return (m ** e) % n

	# Implements efficient exponentiation algorithm
	r = 1 # Initialize result to 1
	e_bin = bin(e)[2:]
	for i in e_bin:
		r = (r**2) % n
		if (i == '1'):
			r = (r*m) % n
	return r

# Modular Inverse Finder
def inverse_finder(a, n):
	isCoprime = (gcd(a,n) == 1)
	if (not isCoprime):
		raise ValueError("a and n are not coprime!")

	_, x, y = gcdExtended(a, n)
	result = (x % n + n) % n
	return result

# Test exponentiation
expResult = exponentiation(2000, 2020, 2023)
print(f"Exponentiation result: {expResult}")

# Test inverse_finder
invResult = inverse_finder(2000, 2023)
print(f"Modular Inverse Finder result: {invResult}")

Exponentiation result: 72
Modular Inverse Finder result: 1935


## 2. Implementing RSA

### a. Key Generation

In [52]:
# Generate a random prime number between a lower and upper bound
# See: https://stackoverflow.com/questions/21043075/generating-large-prime-numbers-in-python
def rand_prime(bits):
    while True:
        p = secrets.randbits(bits)
        if (sympy.isprime(p)):
            break
    return p

# RSA Keygen
def RSA_key_generate(e=65537):
    p = 0
    q = 0
    phi = 0

    # Check if totient(p*q) and e are coprime
    while (gcd(e, phi) != 1) or (e >= phi):
        # Securely pick primes p and q
        p = rand_prime(512)
        q = rand_prime(512)
        while (p == q):
            q = rand_prime(512)
        phi = (p-1)*(q-1)
        #print(f"Primes: {p}, {q}")
    
    # Output/return keys
    n = p*q
    d = inverse_finder(e, phi)
    ke = (e, n)
    kd = (d, n)
    return ke, kd

# Test RSA keygen
print("RSA Keygen Tests:")
for i in range(10):
    ke, kd = RSA_key_generate(e=3)
    print(f"Public Key #{i+1}: {ke}")
    print(f"Private Key #{i+1}: {kd}")

RSA Keygen Tests:
Public Key #1: (3, 35818356531785414241830745226716271482263400438399219906655860165604877080779019779084935268043960639662987070447472874789434045458767486978167828924670450733248833766086674299198884544981605749635477489249951473595046846266665834260369872443218082357633197123363941590163502375964014951509630377625592174387)
Private Key #1: (23878904354523609494553830151144180988175600292266146604437240110403251387186013186056623512029307093108658046964981916526289363639178324652111885949780292326260380949884731878190358200801991044461170270319461595239655188165048724055916552715803485857595096129737064661928750472304939378312598967766805573691, 35818356531785414241830745226716271482263400438399219906655860165604877080779019779084935268043960639662987070447472874789434045458767486978167828924670450733248833766086674299198884544981605749635477489249951473595046846266665834260369872443218082357633197123363941590163502375964014951509630377625592174387)
Public Key #2: (

### b. RSA Encryption

In [53]:
# RSA Encryption
def RSA_encrypt(m, ke):
    msg = int.from_bytes(bytes(m, 'utf-8'), 'little')
    e, n = ke[0], ke[1]
    c = exponentiation(msg, e, n)
    return c

### c. RSA Decryption

In [54]:
# RSA Decryption
def RSA_decrypt(c, kd):
    d, n = kd[0], kd[1]
    m = exponentiation(c, d, n)
    msg = m.to_bytes((m.bit_length() + 7) // 8, 'little').decode('utf-8')
    return msg

# Test RSA Encryption/Decryption
e = 65537
ke, kd = RSA_key_generate(e)
      
message = "123456789"
print(f"Original message: {message}")
ciphertext = RSA_encrypt(message, ke)
plaintext = RSA_decrypt(ciphertext, kd)
print(f"Decrypted plaintext: {plaintext}")

message = "hello world!"
print(f"Original message: {message}")
ciphertext = RSA_encrypt(message, ke)
plaintext = RSA_decrypt(ciphertext, kd)
print(f"Decrypted plaintext: {plaintext}")

Original message: 123456789
Decrypted plaintext: 123456789
Original message: hello world!
Decrypted plaintext: hello world!


## 3. Pohlig-Hellman

### a. Primitive Root

In [55]:
# Check if number is a primitive root of a modulus
def isPrimitiveRoot(g, p):
    # Generate all powers of g from 0 to p-1
    results = []
    for i in range(0, p):
        results.append(g**(i) % p)
        
    # Create set of results and expected powers
    results.sort()
    results = list(set(results))
    expected = np.arange(1, p)
    
    # Check if results enumerate all expected powers
    flag = False
    if (np.array_equal(results, expected)):
        flag = True
    return flag

test1 = isPrimitiveRoot(2, 13)
test2 = isPrimitiveRoot(3, 13)
print(f"Is 2 a primitive root of 13? {test1}")
print(f"Is 3 a primitive root of 13? {test2}")

Is 2 a primitive root of 13? True
Is 3 a primitive root of 13? False


### b. Pohlig-Hellman Algorithm
Given: $2^x \equiv 12 (\text{mod } 13)$

Executing the Pohlig-Hellman Algorithm:
- $a = 2, b = 12, p = 13$
- $(p-1) = 12 = 2^2 + 3^1$
    - $q_1 = 2, q_2 = 3$
    - $r_1 = 2, r_2 = 1$

|q|e|$$g^{\dfrac{p-1}{q^e}}$$|$$h^{\dfrac{p-1}{q^e}}$$|
|-|-|-----------------|-----------------|
|2|2|8|1728|
|3|1|16|20736|

We can first solve for $x_{q2} = x_0 + 2x_1$ and then for $x_{q3} = x_0$.

Solving for $x = x_0 + 2x_1 (mod 2^2)$:
- $b^{\frac{p-1}{q}} \equiv a^{x_0 \frac{p-1}{q}} (\text{mod } p)$
    - $2^{6x_0} \equiv 12^6 \equiv 1 (\text{mod } 13)$
    - $x_0 = 0$
- $(a^{-x_0}b)^{\frac{p-1}{q^2}} \equiv a^{x_1 \cdot \frac{p-1}{q}} (\text{mod } p)$
    - $b^{\frac{p-1}{q^2}} \equiv a^{x_1 \cdot \frac{p-1}{q}} (\text{mod } p)$
    - $12^3 \equiv 2^{6x_1} (\text{mod } 13)$
    - $12 \equiv 2^{6x_1} (\text{mod } 13)$
    - $x_1 = 1$
- Combining coefficients: $x \equiv x_0 + 2x_1 (\text{mod } 2^2) = 2 (\text{mod }4)$

Solving for $x = x_0 (\text{mod }3)$:
- $b^{\frac{p-1}{q}} \equiv a^{x_0 \frac{p-1}{q}} (\text{mod } p)$
    - $12^4 \equiv 2^{4x_0} (\text{mod } 13)$
    - $1 \equiv 2^{4x_0} (\text{mod } 13)$
    - $x_0 = 0$
$x \equiv x_0 = 0 (\text{mod }3)$

We can use the Chinese Remainder Thoerem to solve for x:
- $x = 2(\text{mod }4)$
- $x = 0(\text{mod }3)$
- $x = 3x_1 + 4x_2$
    - $3x_1 = 2(\text{mod }4)$
    - $x_1 = 2$
    - $4x_2 = 0(\text{mod }3)$
    - $x_2 = 0$
- $x = 6$
    


### c. Solving the Discrete Log
Given: $2^x \equiv 12 (\text{mod } 3)$

We know that $2^x \equiv 12 (\text{mod } 13)$ for $x = 6$. We also know that 2 is a primitive root mod 13.

From this, we can construct:
- $2^6 \equiv 12 (\text{mod } 13)$
- $2^4 \equiv 3 (\text{mod } 13)$

Therefore, $x = 4$